# Notebook Description and Purpose
Walk through a PDF summarization flow with LangChain + OpenAI: download a PDF, split it into pages, run a direct summary over a chosen page window, and build a map-reduce summary in a target language.

# Cell 3 Code Explanation
Imports the OpenAI chat model, PDF loader, text splitter, and requests. Run this first so later cells have the right classes available.

In [12]:
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import requests

# Cell 5 Code Explanation
Downloads the marketing PDF to `dest_file_path`. Swap `url` or `dest_file_path` if you want to summarize another document.

In [13]:
dest_file_path = "../../data/principles_of_marketing_book.pdf"
# Download the PDF from blob storage:
url = (
    "https://storage.googleapis.com/strapi_cms_assets/principles_of_marketing_book.pdf"
)
response = requests.get(url)
with open(dest_file_path, "wb") as f:
    f.write(response.content)

# Cell 7 Code Explanation
Loads the downloaded PDF and splits it into `pages` using a recursive text splitter. Adjust `chunk_size` if you need bigger or smaller chunks.

In [14]:
# Create a text splitter, load the PDF and split it:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
loader = PyPDFLoader(dest_file_path)
pages = loader.load_and_split()

In [4]:
print(len(pages))

499


In [15]:
# Setup a chat_model in LangChain:
model_name = "gpt-4o-mini"

llm = ChatOpenAI(model=model_name, temperature=0)

# Cell 11 Code Explanation
Sets a page window (`start_page` to `end_page`), builds a simple LCEL chain (`prompt | llm | StrOutputParser`), and summarizes that slice of `pages`. Change the range to target different sections.

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define page range for summarization
start_page = 0
end_page = 20

# Define prompt
prompt = ChatPromptTemplate.from_template("Write a concise summary of the following:\\n\\n{context}")

# Instantiate chain using modern LCEL approach
chain = prompt | llm | StrOutputParser()

# Invoke chain - format pages as context string
context_str = "\\n\\n".join([doc.page_content for doc in pages[start_page:end_page]])
result = chain.invoke({"context": context_str})
print(result)

**Summary of "Principles of Marketing"**

"Principles of Marketing," published by the University of Minnesota, provides a comprehensive overview of marketing concepts, strategies, and practices. The text is structured into 16 chapters, beginning with a definition of marketing as the process of creating, communicating, delivering, and exchanging offerings that hold value for customers and society. 

Key topics include:

1. **Strategic Planning**: Emphasizes the importance of value propositions and organizational objectives.
2. **Consumer Behavior**: Explores factors influencing buying decisions and the decision-making process.
3. **Market Segmentation**: Discusses targeted marketing strategies and positioning.
4. **Creating Offerings**: Details the components of product offerings, including branding and packaging.
5. **Marketing Channels**: Examines how channels create value and the dynamics of supply chains.
6. **Marketing Research**: Outlines the marketing research process and the imp

# Cell 12: Map-Reduce Overview
Map-reduce is a two-stage pipeline for long inputs:
- Map: run the same prompt over many chunks/pages to get partial summaries.
- Reduce: merge those partials into a single, coherent summary.
In the next cell, we map over a page range (`map_start` to `map_end`) in `target_lang`, then reduce the chunk summaries into one language-specific overview. Adjust the range or language to experiment.

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Choose a page range and target language
map_start = 0
map_end = 10
pages_subset = pages[map_start:map_end]
target_lang = "Spanish"

map_template_string = """Given the following pages of a marketing book. Generate a summary in {lang}:
Pages:
{pages}

Summary:
"""

reduce_template_string = """Given the following {lang} summaries of pages of a marketing book, 
generate a high level description of the book in {lang}:
Summaries:
{summaries}
"""

# Map and reduce prompts
MAP_PROMPT = PromptTemplate(input_variables=["pages", "lang"], template=map_template_string)
REDUCE_PROMPT = PromptTemplate(input_variables=["summaries", "lang"], template=reduce_template_string)

# LCEL map/reduce chains
map_chain = MAP_PROMPT | llm | StrOutputParser()
partial_summaries = map_chain.batch([
    {"pages": doc.page_content, "lang": target_lang} for doc in pages_subset
])

reduce_chain = REDUCE_PROMPT | llm | StrOutputParser()
map_reduce_summary = reduce_chain.invoke({"summaries": "\n\n".join(partial_summaries), "lang": target_lang})

print(f"{target_lang} map-reduce summary (pages", map_start, "to", map_end, "):\n")
print(map_reduce_summary)

Spanish map-reduce summary (pages 0 to 10 ):

**Descripción del libro "Principios de Marketing":**

El libro "Principios de Marketing" es una obra integral que explora los fundamentos esenciales del marketing, proporcionando a los lectores una comprensión profunda de cómo conectar con los consumidores y generar valor para las empresas. A lo largo de sus capítulos, se abordan conceptos clave como la segmentación de mercado, el comportamiento del consumidor, el desarrollo de estrategias de producto, y la importancia de la investigación de mercado en la toma de decisiones.

Desde la definición del marketing y su relevancia en el entorno empresarial actual, hasta la planificación estratégica y la creación de ofertas, el texto ofrece un enfoque práctico y teórico que incluye ejemplos y actividades para facilitar el aprendizaje. Se examinan también las dinámicas del marketing digital y el impacto de las redes sociales, así como la gestión de canales de distribución y cadenas de suministro pa

In [25]:
from src.fnUtils import render_markdown
# Display the Spanish map-reduce summary
render_markdown(map_reduce_summary)

> **Descripción del libro "Principios de Marketing":**
> 
> El libro "Principios de Marketing" es una obra integral que explora los fundamentos esenciales del marketing, proporcionando a los lectores una comprensión profunda de cómo conectar con los consumidores y generar valor para las empresas. A lo largo de sus capítulos, se abordan conceptos clave como la segmentación de mercado, el comportamiento del consumidor, el desarrollo de estrategias de producto, y la importancia de la investigación de mercado en la toma de decisiones.
> 
> Desde la definición del marketing y su relevancia en el entorno empresarial actual, hasta la planificación estratégica y la creación de ofertas, el texto ofrece un enfoque práctico y teórico que incluye ejemplos y actividades para facilitar el aprendizaje. Se examinan también las dinámicas del marketing digital y el impacto de las redes sociales, así como la gestión de canales de distribución y cadenas de suministro para maximizar el valor al cliente.
> 
> El libro también profundiza en temas como las comunicaciones de marketing integradas, la ética en las ventas, la satisfacción del cliente y la lealtad, destacando la importancia de adaptarse a un entorno en constante cambio. Con un enfoque en la aplicación práctica de los conceptos, "Principios de Marketing" se presenta como una guía esencial para estudiantes y profesionales que buscan desarrollar estrategias efectivas en el campo del marketing moderno. 
> 
> Además, esta obra está disponible bajo una licencia Creative Commons, lo que permite su uso y adaptación en contextos educativos, promoviendo así el acceso al conocimiento en el ámbito del marketing.